In [22]:
from pymongo import MongoClient
from ConfigParser import SafeConfigParser
from logging import getLogger
from datetime import datetime

def errorCatcher(d, key):
    try:
        d[key]
        return d[key]
    except:
        return 0


class MongoOps(object):

    def __init__(self):
        self.dbClient = MongoClient("localhost", 27017)
        self.db = self.dbClient["polis"]

    def getNodes(self, queryList):
        queriedNodes = self.db.polis_nodes.aggregate([{
            "$match": {
                "type": queryList[0]
            }
        }, {
            "$project": {
                "_id": 0,
                "id": "$id"
            }
        }])
        nodeList = []
        for item in queriedNodes:
            nodeList.append(item["id"])

        return nodeList

    def getEdges(self, queryList, consensus):
        queriedEdges = self.db.polis_edges.aggregate([{
            "$match": {
                "type": queryList[0]
            }
        }, {
            "$project": {
                "_id": 0,
                "source": "$source",
                "target": "$target",
                "weight": "$weight"
            }
        }])

        edgeList = []
        for item in queriedEdges:
            edgeList.append((item["source"], item["target"], errorCatcher(item["weight"], consensus)))

        return edgeList

In [23]:
ops = MongoOps()
queryList = ["opinion"]
# queriedNodes = ops.getNodes(queryList)
queriedEdges_agree = ops.getEdges(queryList, 'agree')
queriedEdges_disagree = ops.getEdges(queryList, 'disagree')
queriedEdges_moderate = ops.getEdges(queryList, 'moderate')

In [24]:
import graph_tool.all as gt
import itertools
import time
from logging import getLogger
from operator import itemgetter
g = gt.Graph()

In [57]:
iedges_agree = [[int(e[0]), int(e[1])] for e in queriedEdges_agree]
g.add_edge_list(iedges_agree)

iedges_disagree = [[int(e[0]), int(e[1])] for e in queriedEdges_disagree]
# g.add_edge_list(iedges_disagree)

iedges_moderate = [[int(e[0]), int(e[1])] for e in queriedEdges_moderate]
# g.add_edge_list(iedges_moderate)

In [55]:
whole_list = iedges_agree + iedges_disagree + iedges_moderate
unique_edges = [set(tuple(x)) for x in whole_list]

<Graph object, directed, with 692 vertices and 101719 edges at 0x1026c23d0>

In [59]:
edge_weight = g.new_edge_property("vector<int64_t>")

for e in queriedEdges_agree:
    edge_weight[g.edge(e[0], e[1])] = [e[2], 0, 0]
    
for e in queriedEdges_disagree:
    edge_weight[g.edge(e[0], e[1])][1] = e[2]
    
for e in queriedEdges_moderate:
    edge_weight[g.edge(e[0], e[1])][2] = e[2]

In [110]:
# u = gt.GraphView(g, efilt=lambda x: unary(x), directed=True)
# gc = gt.Graph(u, prune=True)

l = gt.label_largest_component(gc, directed=False)
uu = gt.GraphView(g, vfilt=(l.a == False), directed=True)
gcc = gt.Graph(uu, prune=True)

In [124]:
# pos = gt.sfdp_layout(gc)
ebet = gt.betweenness(gc)[1]
gt.graph_draw(gc, pos=pos, edge_color=ebet)

(<PropertyMap object with key type 'Vertex' and value type 'vector<double>', for Graph 0x130fb99d0, at 0x130fb9d90>,
 <PropertyMap object with key type 'Vertex' and value type 'bool', for Graph 0x130fb99d0, at 0x1301de210>)

In [122]:
def unary(x):
    try:
        cond = edge_weight[x].a[0] > 10 and edge_weight[x].a[1] > 10
        return cond
    except:
        print 'catched error!'
        return False
    
def edgeColor(x):
    print x
    if edge_weight[x].a[0] > 10:
        return [0.179, 0.203,0.210, 0.8]
    else: 
        return [0., 0.,0., 0.8]

In [111]:
gcc

<Graph object, directed, with 445 vertices and 19982 edges at 0x130fafa90>

In [ ]:
unary(g.edge(1,0))

In [ ]:
edge_weight

In [ ]:
print g.edge(0,1)

In [ ]:
import numpy as np

In [ ]:
np.ndarray(edge_weight, dtype=float, order='F')